# Send a trigger to Azure Logic Apps to initiate a workflow.

In [0]:
%run "../utils/mount_configuration"

In [0]:
# Load evaluation date from 'tickers_quality_evaluation' task
evaluation_date = dbutils.jobs.taskValues.get(taskKey="tickers_quality_evaluation", key="evaluation_date", debugValue="2025-03-17")

In [0]:
from pyspark.sql.functions import col

warren_buffet_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/warren_buffet_stocks_selection') \
    .filter(col('date') == evaluation_date)

# display(warren_buffet_stocks_selection_df)

In [0]:
benjamin_graham_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/benjamin_graham_stocks_selection') \
    .filter(col('date') == evaluation_date)

# display(benjamin_graham_stocks_selection_df)

In [0]:
def generate_html_table(df):
    
    # Converting DataFrame to a list of dictionaries
    data_dict = df.toPandas().to_dict(orient="records")

    # Building HTML for the table
    table_html = """
    <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;">
      <thead style="background-color: #f2f2f2;">
        <tr>
    """
    # Getting column names from the DataFrame
    columns = df.columns

    # Adding headers
    for col in columns:
        table_html += f"<th>{col}</th>"

    table_html += "</tr></thead><tbody>"

    # Adding rows
    for row in data_dict:
        table_html += "<tr>"
        for col in columns:
            table_html += f"<td>{row[col]}</td>"
        table_html += "</tr>"

    table_html += "</tbody></table>"
    
    return table_html

In [0]:
warren_buffet_table = generate_html_table(warren_buffet_stocks_selection_df)
benjamin_graham_table = generate_html_table(benjamin_graham_stocks_selection_df) 

In [0]:
import requests

# Define the endpoint URL for triggering the Azure Logic App via HTTP request.
logic_app_url = "https://prod-37.eastus.logic.azure.com:443/workflows/3fca59a482dd438db33c987486074907/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=6rPB9fCSmcQJX9Vn8llFdUEmFnLuSPAaZKCxUCJ41GU"


# Prepare the JSON payload
payload = {
  "folderPath": "gold/investment_recommendations/",
  "evaluationDate": f"{evaluation_date}",
  "email_content": f"""
          <html>
          <body style="font-family: Arial, sans-serif; font-size: 14px; color: #333;">
            <p>Hej,</p>
            <p>Poniżej znajdziesz zestawienie wyników analizy firm według kryteriów Warrena Buffetta:</p>
            <h3 style="color:#2F5496;">📊 Tabela: Analiza firm według wybranych wskaźników</h3>
            {warren_buffet_table}
            <br/>
            <br/>
            <p>Poniżej znajdziesz zestawienie wyników analizy firm według kryteriów Benjamin Grahama:</p>
            <h3 style="color:#2F5496;">📊 Tabela: Analiza firm według wybranych wskaźników</h3>
            {benjamin_graham_table}
            <br/>
            <br/>
            <p>Pozdrawiam,<br/>📡 System Analityczny</p>
          </body>
        </html>
  """
}

response = requests.post(logic_app_url, json=payload)

print(f"Status code: {response.status_code}")